This notebook is a template to prepare the input aperture corners for apodised photometry. This means you will likely need to adapt the code for your own purposes.

You will need to provide a table of apertures with columns "RA_i", "Dec_i" in degrees for each ith corner of the aperture, as well as a column with the source "ID". If multiple observations are made for each ID, you can optionally only list the ID in the first instance, provided each observation is grouped together.

You will also need to provide a table of each source with "RA", "Dec" and "ID" columns.

You may find commands like ":%s/\<search\>/\<replace\>/g" in Vim or "esc+x; string-replace; \<search\>; \<replace\>" in eMACS useful in preparing these files.

In [ ]:
print("Preparing environment...")

import astropy.table as tbl
import numpy as np
import platform

print("Done.")

In [ ]:
def GetDirStruct():
    match platform.system():
        case "Windows":
            dirPrefix = "D:/"
        case "Linux" | "Ubuntu":
            dirPrefix = "/mnt/d/"
        case "macOS" | "Darwin":
            dirPrefix = "/Volumes/Storage/"
        case _:
            raise Exception(f"OS not recognised: \"{platform.system()}\". Please define a custom switch inside GetDirStruct().")
    return dirPrefix

sourceDir = "HST_Deep/for_emma_v2a_clean2/prism_trial_01_v2a_clean2/m_check_output/triple_1/"
projDir = "NIRSpec/"
outputDir = "Deep/"
Pointings = [7, 8, 13]
output_list = []

dirPrefix = GetDirStruct()

for pointing in Pointings:
    aperture_table = tbl.Table.read(
        f"{dirPrefix}/Astrometry/{sourceDir}/pointing_{pointing}/ds9_slit_regions.txt", format="ascii"
    )
    source_table = tbl.Table.read(
        f"{dirPrefix}/Astrometry/{sourceDir}/pointing_{pointing}/ds9_targ_regions.txt", format="ascii"
    )
    output_table = tbl.join(aperture_table, source_table, keys="ID")
    output_table.add_column(pointing, name="Visit")
    output_list += [output_table]
output_table = tbl.vstack(output_list).group_by("ID")

In [ ]:
output_table

In [ ]:
output_table.write(f'{dirPrefix}/{projDir}/{outputDir}/deep_apertures.fits', overwrite=True)